# Задание 4. Сверточная сеть для классификации заболеваний растений

Используя созданные датасеты, обучите свёрточную нейронную сеть для определения заболевания растения. Используя фиксированную (по свёрточным слоям) архитектуру модели, посмотрите на качество обучения при различных размерах входных изображений и опишите причину наблюдаемой закономерности.

## Памятка для преподавателя

Цели задания:

- Научить оборачивать собственные данные в совместимый с Pytorch ([torch.utils.data.Dataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) )
- Научить анализировать код. В создаваемом `Dataloader` параметра `shuffle=False`, все батчи одинаковы и модель не обучается.
- Продемонстрировать проблему с определением размерности входа Линейного слоя идущего за сверточными

Особенности:
* <font color='red'>В студенческой версии блокнота у train_loader параметр shuffle = False. Студенты должны найти эту ошибку и исправить ее.  </font>

* Так как мы не рассказываем в лекции про GlobalAveragePooling( [nn.AdaptiveAveragePooling](https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveAvgPool2d.html) ) то мы для каждого размера должны создавать новую модель. Что является антипаттерном.

* Предположение что структура сети не должна зависеть от размеров входа не соответствует действительности. Соответственно у разных студентов будут разные лучшие размеры. Зависеть результат будет от того какую структуру модели студент выбрал.

* При максимальном размере (500x500) изображения  большие модели не помещаются в память. Бороться с этим можно уменьшая batch_size

* Низкая точность объясняется малым размером датасета и сложностью задачи.
* Время выполнения задания на GPU < 9 мин
* С учетом вышесказанного задание надо переделать


Создание датасета из набора файлов

Фотографии листьев растений находятся в трех архивах. Загрузим их:

In [ ]:
!wget https://edunet.kea.su/repo/EduNet-web_dependencies/datasets/ibeans/train.zip
!wget https://edunet.kea.su/repo/EduNet-web_dependencies/datasets/ibeans/validation.zip
!wget https://edunet.kea.su/repo/EduNet-web_dependencies/datasets/ibeans/test.zip

clear_output()

и разархивируем

In [ ]:
!unzip train.zip
!unzip validation.zip
!unzip test.zip

clear_output()

Каждый архив содержит папку с тремя подпапками:


*   angular_leaf_spot
*   bean_rust
*   healthy

подпапки соответствует классу (название заболевания или здоровое растение)

Для работы с данными в таком формате удобно использовать класс [torchvision.datasets.ImageFolder](https://pytorch.org/vision/stable/generated/torchvision.datasets.ImageFolder.html)


In [ ]:
from torchvision.datasets import ImageFolder


val_dataset = ImageFolder("validation")
train_dataset = ImageFolder("train")

print("Classes names", val_dataset.classes)

Выведем одно изображение

In [ ]:
img, cls_num = val_dataset[0]
print(f"Size : {img.size} Class: {val_dataset.classes[cls_num]}")  # pillow
display(img)

Создадим загрузчики


In [ ]:
from torch.utils.data import DataLoader

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

Создайте сверточную сеть для классификации

При этом:
* Ограничьте количество сверточных слоев пятью
* Используйте принцип: при увеличении количества фильтров в два раза, пространственные размеры карты признаков так же уменьшаются в два раза.
* Помните о том, что расход памяти связан не только с количеством параметров модели но и с размерами входа. Поэтому при подаче на вход изображений размером 500х500 можно получить ошибку связанную с нехваткой памяти.

In [ ]:
import torch.nn as nn
import torch


class BeanCNN(nn.Module):
    def __init__(self, side_size=500):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1, stride=1),
            MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1, stride=1),
            MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, padding=1, stride=2),
            nn.ReLU(),
            nn.Flatten(),
        )

        out = self.conv(torch.randn((3, side_size, side_size)).unsqueeze(0))

        self.fc = nn.Sequential(
            nn.Linear(int(out.shape[1]), 512), nn.ReLU(), nn.Linear(512, 3)  # n_classes
        )

    def forward(self, x):
        x = self.conv(x)
        scores = self.fc(x)
        return scores







P.s. Нам нужно менять размер изображения в ходе экспериментов, поэтому мы не стали передавать набор трансформаций в конструктор класса датасета. Будем устанавливать это свойство в начале каждого эксперимента.

In [ ]:
import torchvision.transforms as T


def get_transforms(side_size):
    # create set of transforms applied to the images
    return T.Compose(
        [
            T.Resize((side_size, side_size)),
            T.ToTensor(),
            T.Normalize(mean=[0.5183, 0.4845, 0.6570], std=[0.2111, 0.2227, 0.2291]),
        ]
    )

Обучите несколько вариантов модели, обрабатывающих входы различного размера. Допустимо использовать вспомогательные функции (validate, train) из первого задания. Для изменения размера изображений рекомендуется использовать механизм трансформаций PyTorch.

In [ ]:
%%time

from copy import deepcopy

history = []
side_sizes = [500, 256, 128, 64, 32]
best_model = None
max_accuracy = 0
best_size = None
for side_size in side_sizes:
    # Change transform on both datasets
    transform = get_transforms(side_size)
    train_dataset.transform = transform
    val_dataset.transform = transform

    model = BeanCNN(side_size=side_size)
    model.train()
    model.to(device)

    pp = train(
        model, train_loader, val_loader, lr=0.003, epochs=8, title=str(side_size)
    )
    if max(pp.history_dict["Accuracy/val"]) > max_accuracy:
        best_model = deepcopy(model)
        max_accuracy = max(pp.history_dict["Accuracy/val"])
        best_size = side_size

    history.append(pp)

Сравните графики значений accuracy для разных размеров входных изображений:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_xlabel("img_sizes")
ax.set_xticks(side_sizes)
ax.set_ylabel("best acc on test")
ax.grid()

for i, h in enumerate(history):
    x = h.history_dict["Accuracy/val"]
    ax.plot(x, label=side_sizes[i])
ax.legend()
plt.show()

Проверьте результат на тестовом датасете

In [ ]:
test_dataset = ImageFolder("test", transform=get_transforms(best_size))
test_loader = DataLoader(test_dataset, batch_size=batch_size)
accuracy = validate(best_model, test_loader, device)

print(f"Accuracy on TEST {accuracy:.2f}")

Вывод:

1. На совсем маленьких изображениях недостаточно информации для классификации. Пораженных участков попросту не видно.

2. Разрешения 500 .. 128 в целом достаточно. При таких разрешениях точность зависит от выбранной архитектуры. Так как скорость обучения сильно зависит от объема входных данных, целесообразно уменьшать картинку в 2-3 раза.
3. Для корректной работы алгоритма стохастического градиентного спуска пришлось изменить значения параметра shuffle с False на True, чтобы батчи на разных эпохах различались


## Формат результата

Результатом является показание точности сети при различных размерах входных изображений.

Например:

* Размер изображения 32х32 точность 0.6

* Размер изображения 64х64 точность 0.7

* Размер изображения 500х500 точность 0.57
